<h1>Convolutional Neural Networks</h1>
<img src="https://miro.medium.com/max/4348/1*PXworfAP2IombUzBsDMg7Q.png" width="750" align="center">

In this lab we will be constructing and training a "Convolutional Neural Network" aka a neural network that contains convolution kernels with learnable parameters.<br>
We are also going to learn a bit more about Pytorch transforms and how to create save "checkpoints" for our model!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data.dataloader as dataloader
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
#the size of our mini batches
batch_size     = 64
#How many itterations of our dataset
num_epochs     = 5
#optimizer learning rate
learning_rate  = 0.01
#initialise what epoch we start from
start_epoch    = 0
#initialise best valid accuracy 
best_valid_acc = 0
#where to load/save the dataset from 
data_set_root = "data"

#start from a checkpoint or start from scratch?
start_from_checkpoint = False
#A directory to save our model to (will create it if it doesn't exist)
save_dir = 'Models'
#A name for our model!
model_name = 'LeNet5_MNIST'

In [ ]:
#Set device to GPU_indx if GPU is avaliable
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')

<h3> Create a transform for the input data </h3>
As we have seen, we often wish to perform some operations on data before we pass it through our model. Such operations could be, cropping or resizing images, affine transforms and data normalizations. Pytorch's torchvision module has a large number of such "transforms" which can be strung together sequentially using the "Compose" function. <br>

Pytorch's inbuilt datasets take a transform as an input and will apply this transform to the data before passing it to you! This makes preprocessing data really easy! We will see more about data preprocessing in a later lab!

[torchvision.transforms](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [ ]:
#Prepare a composition of transforms
#transforms.Compose will perform the transforms in order
#NOTE: some transform only take in a PIL image, others only a Tensor
#EG Resize and ToTensor take in a PIL Image, Normalize takes in a Tensor
#Refer to documentation
transform = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize([0.1307], [0.308])])

#Note: ToTensor() will scale unit8 and similar type data to a float and re-scale to 0-1
#Note: We are normalizing with the dataset mean and std 

<h3> Create the training, testing and validation data</h3>
When training many machine learning systems it is best practice to have our TOTAL dataset split into three segments, the training set, testing set and validation set. Up until now we have only had a train/test set split and have used the test set to gauge the performance during training. Though for the most "unbiased" results we should really not use our test set until training is done! So if we want to evaluate our model on an "unseen" part of the dataset we need another split - the validation set. <br>
Training set   - the data we train our model on
Validation set - the data we use to gauge model performance during training
Testing set   - the data we use to "rate" our trained model

In [ ]:
#Define our MNIST Datasets
#Can also try with CIFAR10 Dataset
#https://pytorch.org/docs/stable/torchvision/datasets.html#mnist
train_data = datasets.MNIST(data_set_root, train=True, download=True, transform=transform)
test_data = datasets.MNIST(data_set_root, train=False, download=True, transform=transform)

#We are going to split the train dataset into a train and validation set 90/10
validation_split = 0.9

#Determine the number of samples for each split
n_train_examples = int(len(train_data)*validation_split)
n_valid_examples = len(train_data) - n_train_examples

#The function random_split will take our dataset and split it randomly and give us dataset
#that are the sizes we gave it
#Note: we can split it into to more then two pieces!
train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples])

<h3> Check the lengths of all the datasets</h3>

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

<h3> Create the dataloader</h3>

In [ ]:
#Create the training, Validation and Evaluation/Test Datasets
#It is best practice to separate your data into these three Datasets
#Though depending on your task you may only need Training + Evaluation/Test or maybe only a Training set
#(It also depends on how much data you have)
#https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataloader
train_loader = dataloader.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = dataloader.DataLoader(valid_data, batch_size=batch_size)
test_loader  = dataloader.DataLoader(test_data, batch_size=batch_size)

<h2> Create the LeNet5 network</h2>
LeNet5 is a "classic" old convolution neural network (one of the oldest dating back to 1998) we will be creating an implementation of it here! It uses both convolutional layers and linear layers to "learn" features of the image and perform the classification. It also uses "Max Pooling" to downsample the "feature maps" (the 2d hidden layers at the output of a convolutional layer)

[Max Pooling](https://computersciencewiki.org/index.php/Max-pooling_/_Pooling)

In [ ]:
class LeNet(nn.Module):
    def __init__(self, channels_in):
        #Call the __init__ function of the parent nn.module class
        super(LeNet, self).__init__()
        #Define Convolution Layers
        #conv1 6 channels_inx5x5 kernels
        self.conv1 = nn.Conv2d(channels_in, 6, kernel_size=5)
        
        #conv2 16 6x5x5 kernels
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        
        #Define MaxPooling Layers
        #Default Stride is = to kernel_size
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
        #Define Linear/Fully connected/ Dense Layers
        #Input to linear1 is the number of features from previous conv - 16x5x5
        #output of linear1 is 120
        self.linear1 = nn.Linear(16*5*5, 120)
        #output of linear2 is 84
        self.linear2 = nn.Linear(120, 84)
        #output of linear3 is 10
        self.linear3 = nn.Linear(84, 10)
            
    def forward(self, x):
        #Pass input through conv layers
        #x shape is BatchSize-3-32-32
        
        out1 = F.relu(self.conv1(x))
        #out1 shape is BatchSize-6-28-28
        out1 = self.maxpool(out1)
        #out1 shape is BatchSize-6-14-14

        out2 = F.relu(self.conv2(out1))
        #out2 shape is BatchSize-16-10-10
        out2 = self.maxpool(out2)
        #out2 shape is BatchSize-16-5-5

        #Flatten out2 to shape BatchSize-16x5x5
        out2 = out2.view(out2.shape[0], -1)
        
        out3 = F.relu(self.linear1(out2))
        #out3 shape is BatchSize-120
        out4 = F.relu(self.linear2(out3))
        #out4 shape is BatchSize-84
        out5 = self.linear3(out4)
        #out5 shape is BatchSize-10
        return out5

<h3> Create our model and view the ouput! </h3>

In [ ]:
#create a dataloader itterable object
dataiter = iter(train_loader)
#sample from the itterable object
images, labels = dataiter.next()

In [ ]:
#create an instance of our network
#set channels_in to the number of channels of the dataset images
net = LeNet(channels_in = images.shape[1]).to(device)
#view the network
print(net)

In [ ]:
#pass image through network
out = net(images.to(device))
#check output
out.shape

<h3> Set up the optimizer </h3>

In [ ]:
#Pass our network parameters to the optimiser set our lr as the learning_rate
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

In [ ]:
#Define a Cross Entropy Loss
Loss_fun = nn.CrossEntropyLoss()

<h3> Loading Checkpoints</h3>
This bit of code will load the parameters of a model and a optimizer from file if start_from_checkpoint == True. Saving your model parameters during training is a good idea!

In [ ]:
#Create Save Path from save_dir and model_name, we will save and load our checkpoint here
Save_Path = os.path.join(save_dir, model_name + ".pt")

#Create the save directory if it does note exist
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

#Load Checkpoint
if start_from_checkpoint:
    #Check if checkpoint exists
    if os.path.isfile(Save_Path):
        #load Checkpoint
        check_point = torch.load(Save_Path, map_location = "cpu")
        #Checkpoint is saved as a python dictionary
        #https://www.w3schools.com/python/python_dictionaries.asp
        #here we unpack the dictionary to get our previous training states
        net.load_state_dict(check_point['model_state_dict'])
        optimizer.load_state_dict(check_point['optimizer_state_dict'])
        start_epoch = check_point['epoch']
        best_valid_acc = check_point['valid_acc']
        print("Checkpoint loaded, starting from epoch:", start_epoch)
    else:
        #Raise Error if it does not exist
        raise ValueError("Checkpoint Does not exist")
else:
    #If checkpoint does exist and Start_From_Checkpoint = False
    #Raise an error to prevent accidental overwriting
    if os.path.isfile(Save_Path):
        raise ValueError("Warning Checkpoint exists")
    else:
        print("Starting from scratch")

# Define the accuracy calculator

In [ ]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

# Define the training process

In [ ]:
#This function should perform a single training epoch using our training data
def train(net, device, loader, optimizer, Loss_fun, loss_logger):
    
    #initialise counters
    #We'll keep track of both the loss and accuracy
    epoch_loss = 0
    epoch_acc = 0
    
    #Set Network in train mode
    net.train()
    
    for i, (x, y) in enumerate(loader):
        
        #load images and labels to device
        x = x.to(device) # x is the image
        y = y.to(device) # y is the corresponding label
                
        #Forward pass of image through network and get output
        fx = net(x)
        
        #Calculate loss using loss function
        loss = Loss_fun(fx, y)
        
        #calculate the accuracy
        acc = calculate_accuracy(fx, y)

        #Zero Gradents
        optimizer.zero_grad()
        #Backpropagate Gradents
        loss.backward()
        #Do a single optimization step
        optimizer.step()
        
        #create the cumulative sum of the loss and acc
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        #log the loss for plotting
        loss_logger.append(loss.item())

        #clear_output is a handy function from the IPython.display module
        #it simply clears the output of the running cell
                
    #return the avaerage loss and acc from the epoch as well as the logger array       
    return epoch_loss / len(loader), epoch_acc / len(loader), loss_logger

# Define the testing process

In [ ]:
#This function should perform a single evaluation epoch and will be passed our validation or evaluation/test data
#it WILL NOT be used to train out model
def evaluate(net, device, loader, Loss_fun, loss_logger = None):
    
    #initialise counters
    #We'll keep track of both the loss and accuracy
    epoch_loss = 0
    epoch_acc = 0
    
    #Set network in evaluation mode
    #Layers like Dropout will be disabled
    #Layers like Batchnorm will stop calculating running mean and standard deviation
    #and use current stored values
    net.eval()
    
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            
            #load images and labels to device
            x = x.to(device)
            y = y.to(device)
            
            #Forward pass of image through network
            fx = net(x)
            
            #Calculate loss using loss function
            loss = Loss_fun(fx, y)
            
            #calculate the accuracy
            acc = calculate_accuracy(fx, y)
            
            #log the cumulative sum of the loss and acc
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            #log the loss for plotting if we passed a logger to the function
            if not (loss_logger is None):
                loss_logger.append(loss.item())
                
    #return the avaerage loss and acc from the epoch as well as the logger array       
    return epoch_loss / len(loader), epoch_acc / len(loader), loss_logger

# The training process

In [ ]:
#This cell implements our training loop
training_loss_logger = []
validation_loss_logger = []

for epoch in range(start_epoch, num_epochs):
    
    #call the training function and pass training dataloader etc
    train_loss, train_acc, training_loss_logger = train(net, device, train_loader, optimizer, Loss_fun, training_loss_logger)
    
    #call the evaluate function and pass validation dataloader etc
    valid_loss, valid_acc, validation_loss_logger = evaluate(net, device, valid_loader, Loss_fun, validation_loss_logger)

    #If this model has the highest performace on the validation set 
    #then save a checkpoint
    #{} define a dictionary, each entry of the dictionary is indexed with a string
    if (valid_acc > best_valid_acc):
        print("Saving Model")
        torch.save({
            'epoch':                 epoch,
            'model_state_dict':      net.state_dict(),
            'optimizer_state_dict':  optimizer.state_dict(), 
            'train_acc':             train_acc,
            'valid_acc':             valid_acc,
        }, Save_Path)
    
    clear_output(True)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

In [ ]:
plt.figure(figsize = (10,10))
train_x = np.linspace(0, num_epochs, len(training_loss_logger))
plt.plot(train_x, training_loss_logger, c = "y")
valid_x = np.linspace(0, num_epochs, len(validation_loss_logger))
plt.plot(valid_x, validation_loss_logger, c = "k")

plt.title("LeNet")
plt.legend(["Training Loss", "Validation Loss"])

# Evaluate

In [ ]:
#call the evaluate function and pass the evaluation/test dataloader etc
test_loss, test_acc, _ = evaluate(net, device, test_loader, Loss_fun)
print("The average test loss is: %.2f and the total accuracy is: %.2f%%" %(test_loss, test_acc*100))